In [ ]:
# === Импорты и установка окружения ===
!git clone https://github.com/openai/shap-e
%cd shap-e
!pip install -e .
!pip install googletrans==4.0.0-rc1 matplotlib scikit-learn imageio ipywidgets torch

c:\Users\gerpv\Desktop\pizdec\predpp\preddippract\shap-e


fatal: destination path 'shap-e' already exists and is not an empty directory.


Obtaining file:///C:/Users/gerpv/Desktop/pizdec/predpp/preddippract/shap-e
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning https://github.com/openai/CLIP.git to c:\users\gerpv\appdata\local\temp\pip-install-k8q22brt\clip_bd1b5be75ff94d6bb5fa4e91b1afe604
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: shap-e
    Found existing installation: shap-e 0.0.0
    Uninstalling shap-e-0.0.0:
      Successfully uninstalled shap-e-0.0.0
  Running setup.py develop for shap-e


DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

In [ ]:
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from googletrans import Translator
#from google.colab import files
import imageio
import os
import matplotlib.pyplot as plt
import numpy as np
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# === Основной класс генератора ===

class ShapE3DGenerator:
   

    def __init__(self):
        """Инициализация генератора: загрузка моделей и настройка устройства."""
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"✅ Устройство выбрано: {self.device.type.upper()}")

        # Загрузка моделей
        logger.info("Загрузка моделей...")
        self.transmitter = load_model('transmitter', device=self.device)
        self.text_model = load_model('text300M', device=self.device)
        self.diffusion_process = diffusion_from_config(load_config('diffusion'))
        logger.info("✅ Модели успешно загружены.")

        # Инициализация переводчика
        self.translator = Translator()

        # Хранение состояния
        self.current_latents = None
        self.generated_gifs = []
        self.last_prompt = ""

    def translate_prompt(self, russian_text: str) -> str:
        """
        Переводит текстовый запрос с русского языка на английский.
        :param russian_text: Текст на русском языке.
        :return: Переведенный текст на английском.
        """
        try:
            translation = self.translator.translate(russian_text, src='ru', dest='en')
            return translation.text
        except Exception as e:
            logger.error(f"❌ Ошибка перевода: {e}")
            return russian_text  # Возвращает оригинал, если перевод не удался
        
    def generate_3d_model(self,
                          prompt: str,
                          guidance_scale: float = 15.0,
                          batch_size: int = 1,
                          diffusion_steps: int = 64,
                          render_mode: str = 'nerf',
                          image_size: int = 64) -> None:
        """
        Генерирует 3D модель на основе текстового запроса.
        """
        # Округление размера до ближайшего кратного 8
        image_size = ((image_size + 7) // 8) * 8
        logger.info(f"🚀 Начинаем генерацию для запроса: '{prompt}'")

        # Перевод запроса
        english_prompt = self.translate_prompt(prompt)
        self.last_prompt = english_prompt
        logger.info(f"🌐 Переведенный запрос: '{english_prompt}'")

        # Генерация латентных представлений
        try:
            self.current_latents = sample_latents(
                batch_size=batch_size,
                model=self.text_model,
                diffusion=self.diffusion_process,
                guidance_scale=guidance_scale,
                model_kwargs=dict(texts=[english_prompt] * batch_size),
                progress=True,
                clip_denoised=True,
                use_fp16=True,
                use_karras=True,
                karras_steps=diffusion_steps,
                sigma_min=1e-3,
                sigma_max=160,
                s_churn=0,
            ).to(self.device)

            logger.info(f"✅ Латентные представления сгенерированы. Количество: {batch_size}")

            # Создание камер
            cameras = create_pan_cameras(image_size, self.device)

            # Декодирование и визуализация
            self.generated_gifs = []
            for i, latent in enumerate(self.current_latents):
                images = decode_latent_images(
                    self.transmitter,
                    latent.to(self.device),
                    cameras,
                    rendering_mode=render_mode
                )
                self.generated_gifs.append(images)
                display(gif_widget(images))
                logger.info(f"🖼️  GIF #{i+1} сгенерирован и отображен.")

        except Exception as e:
            logger.error(f"❌ Ошибка при генерации: {e}")  

    def save_generated_gif(self, file_name: str = "generated_model.gif") -> None:
        """
        Сохраняет последний сгенерированный GIF.
        """
        if not self.generated_gifs:
            logger.warning("⚠️ Нет сгенерированных GIF для сохранения.")
            return

        try:
            imageio.mimsave(file_name, self.generated_gifs[0], fps=10)
            logger.info(f"💾 GIF успешно сохранен как '{file_name}'")
        except Exception as e:
            logger.error(f"❌ Ошибка при сохранении GIF: {e}")

    def visualize_latent_space(self) -> None:
        """
        Визуализирует распределение значений в латентном пространстве.
        """
        if self.current_latents is None:
            logger.warning("⚠️ Латентное пространство не сгенерировано. Сначала запустите генерацию.")
            return

        latents_np = self.current_latents.cpu().numpy()
        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        fig.suptitle('Анализ Латентного Пространства', fontsize=16)

        # Гистограмма всех значений
        axes[0, 0].hist(latents_np.flatten(), bins=50, color='skyblue', edgecolor='black')
        axes[0, 0].set_title('Общее Распределение')
        axes[0, 0].set_xlabel('Значение')
        axes[0, 0].set_ylabel('Частота')

        # Гистограммы первых 3 компонентов
        for i in range(min(3, latents_np.shape[1])):
            axes[i//2, i%2].hist(latents_np[:, i], bins=50, color='lightcoral', edgecolor='black')
            axes[i//2, i%2].set_title(f'Компонент {i+1}')
            axes[i//2, i%2].set_xlabel('Значение')
            axes[i//2, i%2].set_ylabel('Частота')

        # Убирает лишние подграфики
        if latents_np.shape[1] < 4:
            for j in range(latents_np.shape[1], 4):
                fig.delaxes(axes[j//2, j%2])

        plt.tight_layout()
        plt.show()
    
    # === Интерфейс пользователя ===

def create_user_interface(generator: ShapE3DGenerator):
    """
    Создает графический интерфейс пользователя.
    """
    # --- Виджеты ввода ---
    prompt_input = widgets.Text(value='красный самолет', description='Запрос:', layout=widgets.Layout(width='auto'))
    guidance_slider = widgets.FloatSlider(value=15.0, min=1.0, max=30.0, step=0.1, description='Масштаб:', layout=widgets.Layout(width='auto'))
    render_dropdown = widgets.Dropdown(options=['nerf', 'stf'], value='nerf', description='Рендер:', layout=widgets.Layout(width='auto'))
    size_slider = widgets.IntSlider(value=64, min=32, max=256, step=8, description='Размер:', layout=widgets.Layout(width='auto'))
    steps_slider = widgets.IntSlider(value=64, min=1, max=256, step=1, description='Шаги:', layout=widgets.Layout(width='auto'))
    batch_slider = widgets.IntSlider(value=1, min=1, max=10, step=1, description='Пакет:', layout=widgets.Layout(width='auto'))

    # --- Кнопки ---
    generate_button = widgets.Button(description="Генерировать", button_style='success', layout=widgets.Layout(width='auto'))
    save_button = widgets.Button(description="Сохранить GIF", button_style='info', layout=widgets.Layout(width='auto'))
    histogram_button = widgets.Button(description="Анализ латентных", button_style='warning', layout=widgets.Layout(width='auto'))

    # --- Прогресс-бар ---
    progress_bar = widgets.FloatProgress(value=0, min=0, max=100, description='Прогресс:', bar_style='info', layout=widgets.Layout(width='auto'))

    # --- Область вывода ---
    output_area = widgets.Output(layout=widgets.Layout(border='solid 1px #ccc', padding='10px'))

    # --- Функции обработки кнопок ---
    def on_generate_clicked(b):
        with output_area:
            clear_output(wait=True)
            progress_bar.value = 0
            progress_bar.description = "Генерация..."
            progress_bar.bar_style = 'info'

            prompt = prompt_input.value.strip()
            if not prompt:
                logger.warning("⚠️ Пустой запрос!")
                return

            try:
                generator.generate_3d_model(
                    prompt=prompt,
                    guidance_scale=guidance_slider.value,
                    batch_size=batch_slider.value,
                    diffusion_steps=steps_slider.value,
                    render_mode=render_dropdown.value,
                    image_size=size_slider.value
                )
                progress_bar.value = 100
                progress_bar.description = "Готово!"
                progress_bar.bar_style = 'success'
            except Exception as e:
                logger.error(f"❌ Ошибка: {e}")
                progress_bar.description = "Ошибка!"
                progress_bar.bar_style = 'danger'

    def on_save_clicked(b):
        with output_area:
            clear_output(wait=True)
            generator.save_generated_gif()

    def on_histogram_clicked(b):
        with output_area:
            clear_output(wait=True)
            generator.visualize_latent_space()

    # --- Привязка событий ---
    generate_button.on_click(on_generate_clicked)
    save_button.on_click(on_save_clicked)
    histogram_button.on_click(on_histogram_clicked)

    # --- Компоновка интерфейса ---
    input_group = widgets.VBox([
        widgets.HBox([prompt_input]),
        widgets.HBox([guidance_slider, render_dropdown]),
        widgets.HBox([size_slider, steps_slider]),
        widgets.HBox([batch_slider])
    ], layout=widgets.Layout(border='solid 1px gray', padding='10px'))

    button_group = widgets.HBox([
        generate_button,
        save_button,
        histogram_button
    ], layout=widgets.Layout(padding='5px'))

    # Отображение  интерфейса
    display(input_group, button_group, progress_bar, output_area)

# === Запуск ===

if __name__ == "__main__":
    shap_e_gen = ShapE3DGenerator()
    create_user_interface(shap_e_gen)
    print("🎉 Генератор 3D моделей Shap-E готов к работе!")
    print("👉 Введите запрос на русском языке и нажмите 'Генерировать'.")
  

2025-12-15 16:17:17,166 - INFO - ✅ Устройство выбрано: CUDA
2025-12-15 16:17:17,167 - INFO - Загрузка моделей...
c:\users\gerpv\desktop\pizdec\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
c:\users\gerpv\desktop\pizdec\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
c:\users\gerpv\desktop\pizdec\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
c:\users\gerpv\desktop\pizdec\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is depre

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

c:\users\gerpv\desktop\pizdec\predpp\preddippract\shap-e\shap_e\models\download.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path, map_location=de

FloatProgress(value=0.0, bar_style='info', description='Прогресс:', layout=Layout(width='auto'))

Output(layout=Layout(border_bottom='solid 1px #ccc', border_left='solid 1px #ccc', border_right='solid 1px #cc…

🎉 Генератор 3D моделей Shap-E готов к работе!
👉 Введите запрос на русском языке и нажмите 'Генерировать'.


In [1]:
!git clone https://github.com/openai/shap-e

fatal: destination path 'shap-e' already exists and is not an empty directory.


In [2]:
!pip install pickleshare
!pip install -e .
!pip install googletrans==4.0.0-rc1
!pip install matplotlib
!pip install scikit-learn


DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

Obtaining file:///C:/Users/gerpv/Desktop/pizdec/predpp/preddippract


DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

In [3]:
%cd shap-e

c:\Users\gerpv\Desktop\pizdec\predpp\preddippract\shap-e


In [4]:
!pip install -e .
!pip install googletrans==4.0.0-rc1
!pip install matplotlib
!pip install scikit-learn

Obtaining file:///C:/Users/gerpv/Desktop/pizdec/predpp/preddippract/shap-e
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning https://github.com/openai/CLIP.git to c:\users\gerpv\appdata\local\temp\pip-install-9pt9ge3g\clip_bbcb4575b7c34a67b233a91ca9b2512f
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: shap-e
    Found existing installation: shap-e 0.0.0
    Uninstalling shap-e-0.0.0:
      Successfully uninstalled shap-e-0.0.0
  Running setup.py develop for shap-e


DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\future-1.0.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\hydra_core-0.11.3-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\appdata\local\programs\python\python312\lib\site-packages\omegaconf-1.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\gerpv\a

In [5]:
import torch
import ipywidgets as widgets
from IPython.display import display
#from google.colab import files
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from googletrans import Translator
import imageio
import os
import matplotlib.pyplot as plt
import numpy as np
import json


In [6]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cu118
True


In [7]:
# Выбор устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Вывод сообщения о выбранном устройстве
if device.type == 'cuda':
    print(f"Выбрано устройство: {device.type.upper()} (GPU)")
else:
    print(f"Выбрано устройство: {device.type.upper()} (CPU)")

Выбрано устройство: CUDA (GPU)


In [9]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))
# Создаем объект переводчика
translator = Translator()

# Функция для перевода текста с русского на английский
def translate_to_english(text):
    translation = translator.translate(text, src='ru', dest='en')
    return translation.text

c:\Users\gerpv\Desktop\pizdec\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
c:\Users\gerpv\Desktop\pizdec\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
c:\Users\gerpv\Desktop\pizdec\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
c:\Users\gerpv\Desktop\pizdec\predpp\preddippract\shap-e\shap_e\models\nn\checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
c:\Users\gerpv\Deskt

In [9]:
batch_size = 1
guidance_scale = 15.0
prompt = "green tank"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
).to(device)  # Перемещаем латентные представления на устройство CUDA

  0%|          | 0/64 [00:00<?, ?it/s]

In [10]:
render_mode = 'nerf' 
size = 64 # это размер рендера; более высокие значения займут больше времени для рендера.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent.to(device), cameras, rendering_mode=render_mode)  # Перемещаем латентные представления на устройство CUDA
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAGOKWWKKWWKJWWKGXGCGV2KFXGCFWGKEXGCDW1+DVl+CXF6DWF…

In [10]:
#Обьединение и добавление интерфейса
# Инициализация переводчика
translator = Translator()

# Функция для перевода текста с русского на английский
def translate_to_english(text):
    translation = translator.translate(text, src='ru', dest='en')
    return translation.text

# Функция для округления размера до ближайшего кратного 8
def round_to_nearest_multiple(value, multiple):
    return (value + multiple - 1) // multiple * multiple

# Создание виджетов для ввода параметров
prompt_widget = widgets.Text(value='красный самолет', description='Prompt:', disabled=False)
guidance_scale_widget = widgets.FloatSlider(value=15.0, min=1.0, max=30.0, step=0.1, description='Guidance Scale:', disabled=False)
render_mode_widget = widgets.Dropdown(options=['nerf', 'stf'], value='nerf', description='Render Mode:', disabled=False)
size_widget = widgets.IntSlider(value=64, min=32, max=256, step=8, description='Size:', disabled=False)
diffusion_steps_widget = widgets.IntSlider(value=64, min=1, max=256, step=1, description='Diffusion Steps:', disabled=False)
batch_size_widget = widgets.IntSlider(value=1, min=1, max=10, step=1, description='Batch Size:', disabled=False)

# Создание кнопки для начала генерации
generate_button = widgets.Button(description="Generate")

# Создание кнопки для сохранения GIF
save_button = widgets.Button(description="Save GIF")

# Создание кнопки для отображения гистограммы
histogram_button = widgets.Button(description="Show Histogram")

# Инициализация переменных для хранения сгенерированных изображений и латентных представлений
generated_images = []
generated_latents = None

# Определение функции, которая будет вызываться при нажатии на кнопку генерации
def on_generate_button_clicked(b):
    global generated_images, generated_latents
    prompt = prompt_widget.value
    translated_prompt = translate_to_english(prompt)
    guidance_scale = guidance_scale_widget.value
    render_mode = render_mode_widget.value
    size = round_to_nearest_multiple(size_widget.value, 8)  # Округление размера до ближайшего кратного 8
    diffusion_steps = diffusion_steps_widget.value
    batch_size = batch_size_widget.value

    latents = sample_latents(
        batch_size=batch_size,
        model=model,
        diffusion=diffusion,
        guidance_scale=guidance_scale,
        model_kwargs=dict(texts=[translated_prompt] * batch_size),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=diffusion_steps,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    ).to(device)  # Перемещаем латентные представления на устройство CUDA

    generated_latents = latents

    cameras = create_pan_cameras(size, device)
    generated_images = []
    for i, latent in enumerate(latents):
        images = decode_latent_images(xm, latent.to(device), cameras, rendering_mode=render_mode)  # Перемещаем латентные представления на устройство CUDA
        generated_images.append(images)
        display(gif_widget(images))

# Определение функции, которая будет вызываться при нажатии на кнопку сохранения
def on_save_button_clicked(b):
    if generated_images:
        output_path = 'generated_gif.gif'
        imageio.mimsave(output_path, generated_images[0], fps=10)
        print(f"GIF сохранен в {output_path}")
        # Создание ссылки для скачивания сохраненного GIF
        #files.download(output_path)
    else:
        print("Изображения еще не сгенерированы. Пожалуйста, сначала сгенерируйте изображения.")

# Функция для визуализации латентных представлений
def visualize_latents(latents):
    # Проверка размерности латентных представлений
    if latents.ndimension() == 1:
        # Если латентные представления одномерные, визуализируем их гистограмму
        plt.figure(figsize=(10, 5))
        plt.hist(latents.cpu().numpy(), bins=50, color='blue', alpha=0.7)
        plt.title('Histogram of Latent Representations')
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        plt.grid(True)
        plt.show()
    else:
        # Если латентные представления многомерные, визуализируем гистограммы для каждого компонента
        num_components = min(latents.shape[1], 5)  # Визуализируем первые 5 компонентов
        plt.figure(figsize=(15, 5 * num_components))
        for i in range(num_components):
            plt.subplot(num_components, 1, i + 1)
            plt.hist(latents[:, i].cpu().numpy(), bins=50, color='blue', alpha=0.7)
            plt.title(f'Histogram of Latent Component {i + 1}')
            plt.xlabel('Value')
            plt.ylabel('Frequency')
            plt.grid(True)
        plt.tight_layout()
        plt.show()

# Определение функции, которая будет вызываться при нажатии на кнопку отображения гистограммы
def on_histogram_button_clicked(b):
    if generated_latents is not None:
        visualize_latents(generated_latents)
    else:
        print("Латентные представления еще не сгенерированы. Пожалуйста, сначала сгенерируйте изображения.")

# Привязка функций к кнопкам
generate_button.on_click(on_generate_button_clicked)
save_button.on_click(on_save_button_clicked)
histogram_button.on_click(on_histogram_button_clicked)

# Создание разделов для организации интерфейса
input_section = widgets.VBox([prompt_widget, guidance_scale_widget, render_mode_widget, size_widget, diffusion_steps_widget, batch_size_widget])
button_section = widgets.HBox([generate_button, save_button, histogram_button])

# Отображение виджетов и кнопок
display(input_section, button_section)

  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIYAAOUAAOQAAOMAAOIAAOEAAOAAAN8AAN4AAN0AANwAANsAANoAAN…